In [2]:
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Extração de dados do Censo Superior para anos específicos, mantendo apenas uma série de atributos desejados.

O intuito é selecionar apenas o código da IES e o código de cursos da IES, para que seja feita posteriormente relacionamento com o ENADE e extração de apenas cursos voltados a Computação + Região Nordeste.

In [3]:
def extrair_dados_censo_colab(caminhos_arquivos, atributos_desejados):
    dados_por_ano = {}
    for ano, caminho_arquivo in caminhos_arquivos.items():
        print(f"Processando ano: {ano} do arquivo: {caminho_arquivo}")
        try:
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin1', low_memory=False)
            colunas_existentes = [col for col in atributos_desejados if col in df.columns]
            colunas_faltando = [col for col in atributos_desejados if col not in df.columns]

            if colunas_faltando:
                print(f"Atenção: As seguintes colunas não foram encontradas para {ano}: {colunas_faltando}")
                print(f"Extraindo apenas as colunas disponíveis: {colunas_existentes}")

            df_extraido = df[colunas_existentes].copy()
            dados_por_ano[ano] = df_extraido
            print(f"Dados do Censo para {ano} carregados com sucesso.")

        except FileNotFoundError:
            print(f"Erro: Arquivo não encontrado em {caminho_arquivo}. Por favor, verifique o caminho.")
            continue
        except Exception as e:
            print(f"Erro ao processar o arquivo para o ano {ano}: {e}")
            continue
    return dados_por_ano

def remover_duplicatas_cursos(dados_por_ano, chaves_duplicidade):
    dados_processados = {}
    for ano, df in dados_por_ano.items():
        print(f"\nVerificando duplicatas para o ano: {ano}")
        num_linhas_antes = len(df)
        chaves_validas = [col for col in chaves_duplicidade if col in df.columns]
        if len(chaves_validas) != len(chaves_duplicidade):
            print(f"Atenção: Nem todas as chaves de duplicidade ({chaves_duplicidade}) foram encontradas no DataFrame para {ano}.")
            print(f"Usando apenas as chaves disponíveis para detecção de duplicatas: {chaves_validas}")

        if not chaves_validas:
            print(f"Aviso: Nenhuma chave válida para remoção de duplicatas para o ano {ano}. Nenhuma duplicata será removida.")
            dados_processados[ano] = df.copy() # Retorna uma cópia para evitar modificações indesejadas
            continue

        df_sem_duplicatas = df.drop_duplicates(subset=chaves_validas, keep='first')
        num_linhas_depois = len(df_sem_duplicatas)

        print(f"Linhas antes da remoção de duplicatas: {num_linhas_antes}")
        print(f"Linhas depois da remoção de duplicatas: {num_linhas_depois}")
        print(f"Total de duplicatas removidas: {num_linhas_antes - num_linhas_depois}")

        dados_processados[ano] = df_sem_duplicatas

    return dados_processados

A extração de cursos, nesse momento, é só para validar os cursos presentes em cada ano lá no ENADE.

In [4]:
if __name__ == "__main__":
    atributos_censo = [
        'NU_ANO_CENSO', 'CO_IES', 'NO_IES', 'CO_CURSO', 'NO_CURSO',
        'NO_REGIAO', 'CO_REGIAO', 'NO_UF', 'SG_UF', 'CO_UF'
    ]

    caminhos_arquivos_censo = {
        2014: '/content/drive/MyDrive/EDM 2025.1/DATASETS/CENSO/2014/MICRODADOS_CADASTRO_CURSOS_2014.CSV',
        2017: '/content/drive/MyDrive/EDM 2025.1/DATASETS/CENSO/2017/MICRODADOS_CADASTRO_CURSOS_2017.CSV',
        2021: '/content/drive/MyDrive/EDM 2025.1/DATASETS/CENSO/2021/MICRODADOS_CADASTRO_CURSOS_2021.CSV'
    }

    df_censo_bruto = extrair_dados_censo_colab(caminhos_arquivos_censo, atributos_censo)
    chaves_duplicidade = ['CO_IES', 'CO_CURSO', 'NU_ANO_CENSO']

    df_censo_limpo = remover_duplicatas_cursos(df_censo_bruto, chaves_duplicidade)
    print("\nRemoção de duplicatas concluída.")

Processando ano: 2014 do arquivo: /content/drive/MyDrive/EDM 2025.1/DATASETS/CENSO/2014/MICRODADOS_CADASTRO_CURSOS_2014.CSV
Atenção: As seguintes colunas não foram encontradas para 2014: ['NO_IES']
Extraindo apenas as colunas disponíveis: ['NU_ANO_CENSO', 'CO_IES', 'CO_CURSO', 'NO_CURSO', 'NO_REGIAO', 'CO_REGIAO', 'NO_UF', 'SG_UF', 'CO_UF']
Dados do Censo para 2014 carregados com sucesso.
Processando ano: 2017 do arquivo: /content/drive/MyDrive/EDM 2025.1/DATASETS/CENSO/2017/MICRODADOS_CADASTRO_CURSOS_2017.CSV
Atenção: As seguintes colunas não foram encontradas para 2017: ['NO_IES']
Extraindo apenas as colunas disponíveis: ['NU_ANO_CENSO', 'CO_IES', 'CO_CURSO', 'NO_CURSO', 'NO_REGIAO', 'CO_REGIAO', 'NO_UF', 'SG_UF', 'CO_UF']
Dados do Censo para 2017 carregados com sucesso.
Processando ano: 2021 do arquivo: /content/drive/MyDrive/EDM 2025.1/DATASETS/CENSO/2021/MICRODADOS_CADASTRO_CURSOS_2021.CSV
Atenção: As seguintes colunas não foram encontradas para 2021: ['NO_IES']
Extraindo apenas a

In [8]:
df_2014 = df_censo_limpo[2014]
df_2014

,NU_ANO_CENSO,CO_IES,CO_CURSO,NO_CURSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF
0,2014,699,116776,ENGENHARIA DE ALIMENTOS,Norte,1.0,Rondônia,RO,11.0
1,2014,699,116782,PEDAGOGIA,Norte,1.0,Rondônia,RO,11.0
2,2014,833,27377,ADMINISTRAÇÃO,Norte,1.0,Rondônia,RO,11.0
3,2014,833,5000035,CIÊNCIAS BIOLÓGICAS,Norte,1.0,Rondônia,RO,11.0
4,2014,833,21267,CIÊNCIAS CONTÁBEIS,Norte,1.0,Rondônia,RO,11.0
...,...,...,...,...,...,...,...,...,...
72117,2014,1060,1304482,GESTÃO DE RECURSOS HUMANOS,Centro-Oeste,5.0,Distrito Federal,DF,53.0
72118,2014,1060,1304743,GESTÃO FINANCEIRA,Centro-Oeste,5.0,Distrito Federal,DF,53.0
72119,2014,1060,1183705,GESTÃO PÚBLICA,Centro-Oeste,5.0,Distrito Federal,DF,53.0
72120,2014,1060,1303429,PEDAGOGIA,Centro-Oeste,5.0,Distrito Federal,DF,53.0


In [10]:
df_2017 = df_censo_limpo[2017]
df_2017

,NU_ANO_CENSO,CO_IES,CO_CURSO,NO_CURSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF
0,2017,699,116776,ENGENHARIA DE ALIMENTOS,Norte,1.0,Rondônia,RO,11.0
1,2017,699,116782,PEDAGOGIA,Norte,1.0,Rondônia,RO,11.0
2,2017,833,5000169,EDUCAÇÃO FÍSICA,Norte,1.0,Rondônia,RO,11.0
3,2017,833,98352,GEOGRAFIA,Norte,1.0,Rondônia,RO,11.0
4,2017,833,75021,MATEMÁTICA,Norte,1.0,Rondônia,RO,11.0
...,...,...,...,...,...,...,...,...,...
117619,2017,1538,1210786,ADMINISTRAÇÃO,Centro-Oeste,5.0,Distrito Federal,DF,53.0
117620,2017,1538,1211337,CIÊNCIAS CONTÁBEIS,Centro-Oeste,5.0,Distrito Federal,DF,53.0
117621,2017,1538,1211277,GESTÃO DE RECURSOS HUMANOS,Centro-Oeste,5.0,Distrito Federal,DF,53.0
117622,2017,1538,1211077,GESTÃO PÚBLICA,Centro-Oeste,5.0,Distrito Federal,DF,53.0


No momento, está tudo bem ter dados ausentes em região e adjacentes. O que importa agora é ter os cursos.

In [11]:
df_2021 = df_censo_limpo[2021]
df_2021

,NU_ANO_CENSO,CO_IES,CO_CURSO,NO_CURSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF
0,2021,1,1113617,Administração Pública,NaN,NaN,NaN,NaN,NaN
1,2021,1,114386,Ciências Naturais E Matemática,NaN,NaN,NaN,NaN,NaN
2,2021,1,1357965,Matemática,NaN,NaN,NaN,NaN,NaN
3,2021,1,105652,Pedagogia,NaN,NaN,NaN,NaN,NaN
4,2021,1,1425400,Tecnologia Educacional,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
444614,2021,1783,95938,Direito,Sul,4.0,Santa Catarina,SC,42.0
444615,2021,1783,1029922,Educação Física,Sul,4.0,Santa Catarina,SC,42.0
444616,2021,1783,1185137,Engenharia Ambiental E Sanitária,Sul,4.0,Santa Catarina,SC,42.0
444617,2021,1783,100493,Pedagogia,Sul,4.0,Santa Catarina,SC,42.0


In [12]:
if 2021 in df_censo_limpo:
        print("\nExemplo de dados limpos para 2021 (primeiras 5 linhas):")
        print(df_censo_limpo[2021].head())


for ano, df_limpo in df_censo_limpo.items():
        output_path = f'/content/drive/MyDrive/EDM 2025.1/Censo_Cursos_{ano}.csv' # Adapte o caminho de saída
        df_limpo.to_csv(output_path, index=False, encoding='utf-8')
        print(f"DataFrame limpo para {ano} salvo em {output_path}")


Exemplo de dados limpos para 2021 (primeiras 5 linhas):
   NU_ANO_CENSO  CO_IES  CO_CURSO                        NO_CURSO NO_REGIAO  \
0          2021       1   1113617           Administração Pública       NaN   
1          2021       1    114386  Ciências Naturais E Matemática       NaN   
2          2021       1   1357965                      Matemática       NaN   
3          2021       1    105652                       Pedagogia       NaN   
4          2021       1   1425400          Tecnologia Educacional       NaN   

   CO_REGIAO NO_UF SG_UF  CO_UF  
0        NaN   NaN   NaN    NaN  
1        NaN   NaN   NaN    NaN  
2        NaN   NaN   NaN    NaN  
3        NaN   NaN   NaN    NaN  
4        NaN   NaN   NaN    NaN  
DataFrame limpo para 2014 salvo em /content/drive/MyDrive/EDM 2025.1/Censo_Cursos_2014.csv
DataFrame limpo para 2017 salvo em /content/drive/MyDrive/EDM 2025.1/Censo_Cursos_2017.csv
DataFrame limpo para 2021 salvo em /content/drive/MyDrive/EDM 2025.1/Censo_Cursos_